In [12]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from time import sleep


In [13]:
# get dates range for articles and change date format 


base_url = "https://www.npr.org/sections/politics/archive?start=placeholder&date="

def get_all_dates(start_date, end_date):

    date_format = "%-m-%d-%Y"

    date_strings = []

    current = start_date
    while current <= end_date:
        format_date = current.strftime(date_format)

        date_strings.append(format_date)

        current += timedelta(days=1)

    return date_strings 



    


In [14]:
#test getting url html for one day

sample = "https://www.npr.org/sections/politics/archive?start=placeholder&date=6-21-2025"

response = requests.get(sample)

if response.status_code == 200:

    print('sucess')
else:
    print('Failed to retrieve the webpage. Status code:', response.status_code)



sucess


In [15]:

# get html for a given url
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

html = get_html("https://www.npr.org/sections/politics/archive?start=placeholder&date=6-21-2025")


In [16]:
# parse articles and get title, link and date from each article on page

def parse_articles(html):
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all('article')

    if not articles:
        return None
    
    parsed_articles = []
    for article in articles:
        title_tag = article.find('h2', class_='title')
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        link = title_tag.find('a')['href']
        date_tag = article.find('time')
        date = date_tag.get('datetime')
        
        parsed_articles.append({
            'title': title,
            'link': link,
            'date': date
        })
        
    return parsed_articles
    
parsed_articles = parse_articles(html)
#print(parsed_articles)
#print(len(parsed_articles))

In [17]:

# from link url get article content 

def get_article_content(story_link):
    html_content = get_html(story_link)
    if not html_content:
        return None

    soup = BeautifulSoup(html_content, 'html.parser')
    storytext = soup.find(id='storytext')
    if not storytext:
        return None
    
    content =[]
    paragraphs = storytext.find_all('p')
    for p in paragraphs:
        content.append(p.get_text(strip=True))

    return content


# article_content = get_article_content(parsed_articles[0]['link'])     
# print(article_content)

    
    
    
    

In [18]:

def get_articles_for_date(date,topic):
    '''
    Get articles for date in month day year format
    Args:
        date (str): Date in month day year format
    Returns:
        list: List of articles for date
    '''
    url = f"https://www.npr.org/sections/{topic}/archive?start=placeholder&date={date}"
    html = get_html(url)
    if not html:
        return None
    article_links = parse_articles(html)

    articles_for_date = []

    # get article content
    for article_link in article_links:
        article_date = datetime.strptime(article_link['date'], "%Y-%m-%d")
        passed_date = datetime.strptime(date, "%m-%d-%Y")

        if article_date != passed_date:
            article_links.remove(article_link)
        else:
            article_content = get_article_content(article_link['link'])
            article_link['content'] = article_content
            sleep(1) 
            articles_for_date.append(article_link)

    return articles_for_date
    

# articles = get_articles_for_date("6-21-2025")
# print(articles)
# print(len(articles))


In [19]:
test_pd = pd.DataFrame()
# load datafrom from articles variable

test_pd = pd.concat([test_pd, pd.DataFrame(articles)])

#print(test_pd.head())
#print(test_pd.shape)

#print(len(articles))

    

NameError: name 'articles' is not defined

In [24]:
# checking how many articles we get for each date

start_date = datetime(2025, 4, 1)
end_date = datetime(2025, 7, 1)

topics = ["politics", "climate", "health", 'science']

dates = get_all_dates(start_date, end_date)

npr_articles = pd.DataFrame()


for topic in topics:
    for date in dates:
        print("Getting articles for date: ", date, "for topic: ", topic)
        date_articles = get_articles_for_date(date, topic)
        print("Number of articles for date: ", len(date_articles))
        npr_articles = pd.concat([npr_articles, pd.DataFrame(date_articles)])

# save dataframe to pickle file
    npr_articles.to_pickle(f"npr_{topic}_articles{start_date.strftime('%m-%d-%Y')}.pkl")
    npr_articles = pd.DataFrame()



    

Getting articles for date:  4-01-2025 for topic:  politics
Number of articles for date:  13
Getting articles for date:  4-02-2025 for topic:  politics
Number of articles for date:  15
Getting articles for date:  4-03-2025 for topic:  politics
Number of articles for date:  11
Getting articles for date:  4-04-2025 for topic:  politics
Number of articles for date:  10
Getting articles for date:  4-05-2025 for topic:  politics
Number of articles for date:  3
Getting articles for date:  4-06-2025 for topic:  politics
Number of articles for date:  1
Getting articles for date:  4-07-2025 for topic:  politics
Number of articles for date:  13
Getting articles for date:  4-08-2025 for topic:  politics
Number of articles for date:  15
Getting articles for date:  4-09-2025 for topic:  politics
Number of articles for date:  11
Getting articles for date:  4-10-2025 for topic:  politics
Number of articles for date:  6
Getting articles for date:  4-11-2025 for topic:  politics
Number of articles for d

In [ ]:
# to check dataframe 


# print(npr_politics_articles.head())
# print(npr_politics_articles.shape)

# print(npr_politics_articles['date'].unique())